In [1]:
import os
import pandas as pd

In [ ]:
os.getcwd()

In [ ]:
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')
mushrooms.head()

In [ ]:
mushrooms.dtypes

In [ ]:
n_rows = mushrooms.shape[0]
n_columns = mushrooms.shape[1]

print(f'There are {n_rows} rows and {n_columns} columns.')

In [ ]:
print("There are no missing values." if not mushrooms.isnull().any().any()\
      else "There are missing values in the dataframe.")

In [ ]:
nan_counts = mushrooms.isna().sum()
nan_counts[nan_counts > 0]

In [ ]:
edible_count = (mushrooms['class'] == 'e').sum()
poisonous_count = mushrooms.shape[0] - edible_count
print(f"Number of edible mushrooms: {edible_count}\nNumber of poisonous mushrooms: {poisonous_count}")

In [ ]:
import pandas as pd
import numpy as np
from UDFs import DecisionTreeClassifier, compute_accuracy

mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

mushrooms_1 = mushrooms.dropna(axis=1)

X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

X[:, 0]

In [1]:
import pandas as pd
import numpy as np
from UDFs import DecisionTreeClassifier, compute_accuracy

mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

#mushrooms_1 = mushrooms.dropna(axis=1)

mushrooms_1 = mushrooms.copy()

X_names = mushrooms_1.drop('class', axis=1).columns
X_names

Index(['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [2]:
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

tree = DecisionTreeClassifier(max_depth=3, n_features=3, n_quantiles=5)
tree.fit(X, y)

predictions = tree.predict(X)

accuracy = compute_accuracy(y, predictions)
print(f"Accuracy: {accuracy:.2f}")

Fitting the model...
Considering thresholds for feature index 12: ['b' 'e' 'f' 'g' 'k' 'l' 'n' 'o' 'p' 'r' 'u' 'w' 'y']
Considering thresholds for feature index 4: ['f' 't']


TypeError: unsupported operand type(s) for -: 'str' and 'str'